# Salvar e carregar modelos

O progresso do modelo pode ser salvo durante ou após o treino. Isso quer dizer que um modelo pode retomar o treino de onde parou e evitar que fique em treinamento por muito tempo. Salvar o modelo significa também que podemos compartilhá-lo para que outros possam usá-lo ou até melhorá-lo. Ao publicar o modelo e as técnicas de pesquisa, a maioria dos estudantes de aprendizado de máquina têm acesso a:

- O código para criar o modelo e;

- Os pesos treinados, ou parâmetros, para o modelo.

Compartilhar essas informações auxilia outras pessoas a entender como o modelo funciona e a utilização com outros dados.

## Opções

Há diversas formas de salvar os modelos do TensorFlow, dependendo da API usada. Usaremos a API do [tf.keras](https://www.tensorflow.org/guide/keras?hl=pt-br), logo veremos o passo a passo para salvar o modelo baseado nessa API. Para outras APIs, acesse o guia [Salvar e restaurar](https://www.tensorflow.org/guide/saved_model?hl=pt-br) ou [Salvar em arquivos](https://www.tensorflow.org/guide/basics?hl=pt-br#object-based_saving).

## Configurar

### Instalar e importar

Vamos importar o TensorFlow:

In [14]:
# Importar as bibliotecas
import os
import tensorflow as tf
from tensorflow import keras

# Mostrar a versão do TensorFlow
print(tf.__version__)

2.8.2


### Obter um conjunto de dados de exemplo

Para este estudo de como salvar e carregar os pesos de um modelo, usaremos o conjunto e dados [MNIST](http://yann.lecun.com/exdb/mnist/). Para ficar mais didático o estudo, usaremos somente os mil primeiros exemplos:

In [15]:
# Abrir o conjunto de dados e selecionar os mil primeiros exemplos
(treino_imagens, treino_rotulos), (teste_imagens, teste_rotulos) = tf.keras.datasets.mnist.load_data()
treino_rotulos = treino_rotulos[:1000]
teste_rotulos = teste_rotulos[:1000]
treino_imagens = treino_imagens[:1000].reshape(-1, 28*28)/255.0
teste_imagens = teste_imagens[:1000].reshape(-1, 28*28)/255.0

### Definir um modelo

Começaremos com a construção de um modelo sequencial simples:

In [16]:
# Definir o modelo sequencial simples
def criar_modelo():
  modelo = tf.keras.models.Sequential([
            keras.layers.Dense(512,
                               activation='elu',
                               input_shape=(784,)),
            keras.layers.Dropout(0.2),
            keras.layers.Dense(10)
  ])

  modelo.compile(optimizer='adam',
                 loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=[tf.metrics.SparseCategoricalAccuracy()])
  
  return modelo

In [17]:
# Criar a instância do modelo
modelo = criar_modelo()

# Mostrar a arquitetura do modelo
modelo.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Salvar pontos de verificação durante o treino

Podemos usar um modelo treinado sem precisar retreiná-lo ou retomar o treino de onde parou, caso o treino tenha sido interrompido. A função [tf.keras.callbacks.ModelCheckpoint](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint?hl=pt-br) permite que o modelo seja salvo *durante* e ao *final* do treino.

### Utilização de retorno de chamada do ponto de verificação

Criaremos um retorno de chamada [tf.keras.callbacks.ModelCheckpoint](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint?hl=pt-br) que economize pesos apenas no treino:

In [18]:
# Criar um diretório para armazenar o documento com o checkpoint
caminho_checkpoint = 'treino_1/cp.ckpt'
diretorio_checkpoint = os.path.dirname(caminho_checkpoint)

# Criar um callback que salva os pesos do modelo
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=caminho_checkpoint,
                                                 save_weights_only=True,
                                                 verbose=1)

# Treinar o modelo com o novo callback
modelo.fit(treino_imagens,
           treino_rotulos,
           epochs=10,
           validation_data=(teste_imagens, teste_rotulos),
           callbacks=[cp_callback])

Epoch 1/10
30/32 [===========================>..] - ETA: 0s - loss: 1.0248 - sparse_categorical_accuracy: 0.6740
Epoch 1: saving model to treino_1/cp.ckpt
32/32 [==============================] - 1s 13ms/step - loss: 1.0056 - sparse_categorical_accuracy: 0.6800 - val_loss: 0.6294 - val_sparse_categorical_accuracy: 0.7990
Epoch 2/10
24/32 [=====================>........] - ETA: 0s - loss: 0.3818 - sparse_categorical_accuracy: 0.8958
Epoch 2: saving model to treino_1/cp.ckpt
32/32 [==============================] - 0s 8ms/step - loss: 0.3956 - sparse_categorical_accuracy: 0.8900 - val_loss: 0.5524 - val_sparse_categorical_accuracy: 0.8280
Epoch 3/10
24/32 [=====================>........] - ETA: 0s - loss: 0.2959 - sparse_categorical_accuracy: 0.9258
Epoch 3: saving model to treino_1/cp.ckpt
32/32 [==============================] - 0s 8ms/step - loss: 0.2970 - sparse_categorical_accuracy: 0.9230 - val_loss: 0.5402 - val_sparse_categorical_accuracy: 0.8290
Epoch 4/10
32/32 [===============

Isso cria uma única coleção de arquivo de checkpoint do TensorFlow que são atualizados no final de cada época:

In [19]:
# Ver os documentos presentes no diretório
os.listdir(diretorio_checkpoint)

['cp.ckpt.index', 'checkpoint', 'cp.ckpt.data-00000-of-00001']

Desde que dois modelos compartilhem a mesma arquitetura, podemos compartilhar os pesos entre eles. Logo, ao restaurar um modelo somente de pesos, criaremos um modelo com a mesma arquitetura do modelo original e, em seguida, definir seus pesos.

Agora reconstruiremos um modelo e não o treinaremos, mas o avaliaremos no conjunto de teste. Um modelo não treinado naão generalizará tão bem:

In [20]:
# Criar um novo modelo sem o treinamento
modelo = criar_modelo()

# Avaliar o modelo não treinado
loss, acc = modelo.evaluate(teste_imagens,
                            teste_rotulos,
                            verbose=2)
print(f'Modelo não treinado: {acc:.2%} de acurácia')

32/32 - 0s - loss: 2.4932 - sparse_categorical_accuracy: 0.0470 - 170ms/epoch - 5ms/step
Modelo não treinado: 4.70% de acurácia


Vamos agora carregar os pesos do modelo treinado:

In [21]:
# Carregar os pesos do modelo treinado no nosso modelo não treinado
modelo.load_weights(caminho_checkpoint)

# Testar o modelo sem o treino, mas com os pesos do modelo treinado
loss, acc = modelo.evaluate(teste_imagens,
                            teste_rotulos,
                            verbose=2)
print(f'Modelo não treinado com pesos do modelo treinado: {acc:.2%} de acurácia')

32/32 - 0s - loss: 0.5408 - sparse_categorical_accuracy: 0.8390 - 76ms/epoch - 2ms/step
Modelo não treinado com pesos do modelo treinado: 83.90% de acurácia


### Opções de retorno de chamada de ponto de verificação

O retorno de chamada oferece diversas opções para fornecer nomes exclusivos para pontos de verificação e ajustar a frequência do ponto de verificação.

Treinaremos um novo modelo e salvaremos o ponto de verificação com nomes exclusivos uma vez a cada 5 épocas:

In [22]:
# Incluir a época no nome do aquivo (usar str.format, tipo um REGEX)
caminho_checkpoint = 'treino_2/cp-{epoch:04d}.ckpt'
diretorio_checkpoint = os.path.dirname(caminho_checkpoint)

tamanho_lote = 32

# Criar um callback que salva os pesos do modelo de 5 em 5 épocas
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=caminho_checkpoint,
    verbose=1,
    save_weights_only=True,
    save_freq=5*tamanho_lote
)

# Criar um novo modelo
modelo = criar_modelo()

# Salvar os pesos usando o formato de "caminho_checkpoint"
modelo.save_weights(caminho_checkpoint.format(epoch=0))

# Treinar o modelo com o novo callback
modelo.fit(
    treino_imagens,
    treino_rotulos,
    epochs=50,
    batch_size=tamanho_lote,
    callbacks=[cp_callback],
    validation_data=(teste_imagens, teste_rotulos),
    verbose=0
)


Epoch 5: saving model to treino_2/cp-0005.ckpt

Epoch 10: saving model to treino_2/cp-0010.ckpt

Epoch 15: saving model to treino_2/cp-0015.ckpt

Epoch 20: saving model to treino_2/cp-0020.ckpt

Epoch 25: saving model to treino_2/cp-0025.ckpt

Epoch 30: saving model to treino_2/cp-0030.ckpt

Epoch 35: saving model to treino_2/cp-0035.ckpt

Epoch 40: saving model to treino_2/cp-0040.ckpt

Epoch 45: saving model to treino_2/cp-0045.ckpt

Epoch 50: saving model to treino_2/cp-0050.ckpt


Note que agora há mais checkpoints no diretório. Escolheremos o mais recente:

In [23]:
# Ver os documentos no diretório
os.listdir(diretorio_checkpoint)

['cp-0025.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0015.ckpt.index',
 'cp-0050.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0040.ckpt.index',
 'checkpoint',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0000.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001']

In [24]:
# Pegar o último checkpoint
ultimo_cp = tf.train.latest_checkpoint(diretorio_checkpoint)
ultimo_cp

'treino_2/cp-0050.ckpt'

Vale lembrar que o TensorFlow salva apenas os 5 checkpoints mais recentes.

Para testar o modelo, carregaremos o último checkpoint e o reavaliaremos:

In [25]:
# Criar um novo modelo
modelo = criar_modelo()

# Carregar o último checkpoint
modelo.load_weights(ultimo_cp)

# Reavaliar o modelo
loss, acc = modelo.evaluate(
    teste_imagens,
    teste_rotulos,
    verbose=2
)
print(f'Modelo não treinado com os pesos do checkpoint: {acc:.2%} de acurácia')

32/32 - 0s - loss: 0.7800 - sparse_categorical_accuracy: 0.8350 - 174ms/epoch - 5ms/step
Modelo não treinado com os pesos do checkpoint: 83.50% de acurácia


## O que são esses arquivos?

O código que vimos acima guarda os pesos em uma coleção de arquivos de formatos de [ponto de verificação](https://www.tensorflow.org/guide/saved_model?hl=pt-br#save_and_restore_variables) que possuem apenas os pesos treinados em formato binário. Os pontos de vericação contêm:

- Um ou mais fragmentos que possuem os pesos do nosso modelo e;

- Um arquivo de índice que informa quais pesos são armazenados em qual parte do modelo.

Se estivermos treinando um uma única máquina, teremos o sufixo do arquivo como: `.data-00000-of-00001`.

## Salvar os pesos manualmente

Salvaremos os pesos manualmente com o método [save_weights](https://www.tensorflow.org/api_docs/python/tf/keras/Model?hl=pt-br#save_weights). Por padrão, [tf.keras](https://www.tensorflow.org/api_docs/python/tf/keras?hl=pt-br) (e `save_weigths`em particular) usa o formato de [checkpoint](https://www.tensorflow.org/guide/checkpoint?hl=pt-br) TensorFlow com a extensão `.ckpt`. Para salvar no formato [HDF5](https://docs.h5py.org/en/stable/), veja o tutorial [Salvar e serializar modelos](https://www.tensorflow.org/guide/keras/save_and_serialize?hl=pt-br#weights-only_saving_in_savedmodel_format):

In [26]:
# Salvar os pesos
modelo.save_weights('./checkpoints/meu_checkpoint')

# Criar um novo modelo
modelo = criar_modelo()

# Reataurar os pesos
modelo.load_weights('./checkpoints/meu_checkpoint')

# Avaliar o modelo
loss, acc = modelo.evaluate(teste_imagens,
                            teste_rotulos,
                            verbose=2)
print(f'Modelo com os pesos carregados: {acc:.2%} de acurácia')

32/32 - 0s - loss: 0.7800 - sparse_categorical_accuracy: 0.8350 - 168ms/epoch - 5ms/step
Modelo com os pesos carregados: 83.50% de acurácia


## Salvar todo o modelo

Chamaremos o método [save](https://www.tensorflow.org/api_docs/python/tf/keras/Model?hl=pt-br#save) para salvar a arquitetura, os pesos e a configuração de treino de um modelo em um único arquivo/pasta. Isso torna possível a exportação do modelo para que possa ser usado sem precisar acessar o código fonte e retreinar o modelo. Como o estado do otimizador é recuperado, podemos retomar o treino exatamente de onde parou.

Um modelo inteiro poderá ser salvo em dois formatos de arquivo diferentes ([SavedModel](https://www.tensorflow.org/js/tutorials/conversion/import_saved_model?hl=pt-br) e [HDF5](https://www.tensorflow.org/js/tutorials/conversion/import_keras?hl=pt-br)). O formato TensorFlow `SavedModel`é o formato de arquivo padrão no TF2.x. Contudo, os modelos podem ser salvos no formato `HDF5`.

Salvar um modelo totalmente funcional é muito útil, pois podemos carregá-lo no TensorFlow.js, treiná-lo e executá-lo em navegadores da Web ou convertê-lo para executar em dispositivos móveis com o [TensorFlow Lite](https://www.tensorflow.org/lite/convert?hl=pt-br#converting_a_keras_model_).

### Formato SavedModel

O formato `SavedModel`é outra maneira de serializar modelos. Os modelos salvos nesse formato podem ser restaurados com o método [tf.keras.models.load_model](https://www.tensorflow.org/api_docs/python/tf/keras/models/load_model?hl=pt-br) e são compatíveis com o [TensorFlow Serving](https://www.tensorflow.org/tfx/guide/serving). O [guia SavedModel](https://www.tensorflow.org/guide/saved_model?hl=pt-br) mostra como servir/inspecionar esse formato. As células abaixo mostram as etapas para salvar e restaurar o modelo:

In [27]:
# Criar e treinar o novo modelo
modelo = criar_modelo()
modelo.fit(treino_imagens,
           treino_rotulos,
           epochs=5)

# Salvar a entrada do modelo como "SavedModel"
!mkdir -p modelo_salvo
modelo.save('modelo_salvo/meu_modelo')

Epoch 1/5
32/32 [==============================] - 1s 4ms/step - loss: 1.0433 - sparse_categorical_accuracy: 0.6660
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4112 - sparse_categorical_accuracy: 0.8770
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2839 - sparse_categorical_accuracy: 0.9210
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2499 - sparse_categorical_accuracy: 0.9310
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1849 - sparse_categorical_accuracy: 0.9580
INFO:tensorflow:Assets written to: modelo_salvo/meu_modelo/assets


O formato `SavedModel`é um diretório que possui um binário *protobuf* e um ponto de verificação do TensorFlow. Vamos inspecionar o diretório do modelo salvo:

In [29]:
# Listar o diretório "modelo_salvo"
os.listdir('modelo_salvo')

['meu_modelo']

In [30]:
# Listar o diretóio "modelo_salvo/meu_modelo"
os.listdir('modelo_salvo/meu_modelo')

['assets', 'keras_metadata.pb', 'saved_model.pb', 'variables']

Vamos carregar um novo modelo Keras do modelo salvo:

In [32]:
# Carregar um novo modelo com o modelo salvo
novo_modelo = tf.keras.models.load_model('modelo_salvo/meu_modelo')

# Verificar a arquitetura
novo_modelo.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 512)               401920    
                                                                 
 dropout_10 (Dropout)        (None, 512)               0         
                                                                 
 dense_21 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


O modelo restaurado é compilado com os mesmos argumentos do modelo original. Vamos executar esse novo modelo e avaliar a sua previsão:

In [33]:
# Avaliar o modelo restaurado
loss, acc = novo_modelo.evaluate(teste_imagens,
                                 teste_rotulos,
                                 verbose=2)
print(f'Modelo restaurado: {acc:.2%} de acurácia')
print(novo_modelo.predict(teste_imagens).shape)

32/32 - 0s - loss: 0.4949 - sparse_categorical_accuracy: 0.8370 - 198ms/epoch - 6ms/step
Modelo restaurado: 83.70% de acurácia
(1000, 10)


### Formato HDF5

A API Keras fornece um formato básico de salvamento quando é usado o padrão [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format):

In [35]:
# Criar e treinar um novo modelo
modelo = criar_modelo()
modelo.fit(treino_imagens,
           treino_rotulos,
           epochs=5)

# Salvar a entrada do modelo em um arquivo HDF5
# A extensão '.h5' indica que o modelo deverá ser salvo como HDF5
modelo.save('meu_modelo.h5')

Epoch 1/5
32/32 [==============================] - 1s 6ms/step - loss: 0.9879 - sparse_categorical_accuracy: 0.6980
Epoch 2/5
32/32 [==============================] - 0s 7ms/step - loss: 0.4090 - sparse_categorical_accuracy: 0.8750
Epoch 3/5
32/32 [==============================] - 0s 7ms/step - loss: 0.2815 - sparse_categorical_accuracy: 0.9200
Epoch 4/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2271 - sparse_categorical_accuracy: 0.9380
Epoch 5/5
32/32 [==============================] - 0s 6ms/step - loss: 0.1752 - sparse_categorical_accuracy: 0.9580


Vamos agora carregar o modelo salvo no arquivo `HDF5`:

In [36]:
# Recriaremos o mesmo modelo, incluindo seus pesos e o otimizador
novo_modelo = tf.keras.models.load_model('meu_modelo.h5')

# Mostrar a arquitetura do modelo
novo_modelo.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 512)               401920    
                                                                 
 dropout_12 (Dropout)        (None, 512)               0         
                                                                 
 dense_25 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Vejamos a sua acurácia:

In [37]:
loss, acc = novo_modelo.evaluate(teste_imagens,
                                 teste_rotulos,
                                 verbose=2)
print(f'Modelo recuperado: {acc:.2%} de acurácia')

32/32 - 0s - loss: 0.4738 - sparse_categorical_accuracy: 0.8560 - 375ms/epoch - 12ms/step
Modelo recuperado: 85.60% de acurácia


O Keras salva modelos inspecionando suas arquiteturas. Usar essas técnicas salva tudo:

- Os valores de peso;

- A arquitetura do modelo;

- A configuração de treino do modelo (as informações que passamos para `.compile()`) e;

- O otimizador e seu estado, quando houver (isso permite continuar o treino de onde parou).

O Keras não pode salvar os otimizadores `v1.x` ([tf.compact.v1.train](https://www.tensorflow.org/api_docs/python/tf/compat/v1/train?hl=pt-br), pois eles não são compatíveis com checkpoints. Para os otimizadores `v1.x`, precisamos recompilar o modelo após o carregamento, perdendo assim o estado do otimizador.

### Salvar objetos personalizados

A principal diferença entre `HDF5`e `SavedModel` é que o primeiro usa as configurações de objeto para salvar a arquitetura do modelo, enquanto o segundo salva o gráfico de execução. Desse modo, `SavedModel` é capaz de salvar objetos personalizados como modelos de subclasses e camadas personalizadas sem exigir o código original.

Para salvar objetos personalizados em `HDF5`, devemos fazer o seguinte:

1. Definir um método **get_config** em nosso objeto e, de modo opcional, um método de **from_config**.
  - **get_config(self)** retorna um dicionário no formato JSON de parâmetros necessários para recriar o objeto.
  - **from_config(cls, config)** usa a configuração retornada de **get_config(self)** para criar um novo objeto. Por padrão, esta função utilizará a configuração como *kwargs* de inicialização (__return cls(**config)__).
2. Passar o objeto para o argumento **custom_objects** ao carregar o modelo. O argumento deverá ser um dicionário mapeando o nome da classe *string* para a classe Python. Por exemplo, **tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer}**).

Acesse o tutorial [Escrevendo camadas e modelo do zero](https://www.tensorflow.org/guide/keras/custom_layers_and_models?hl=pt-br) para obter exemplos de objetos personalizados e **get_config**.